In [1]:
import os 
import sys
import subprocess
import pyspark.sql.column
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from datetime import datetime
import logging
from typing import Callable, List, Dict


from db_connectors import *
from logger import Logger
from spark_profiling_exceptions import *

In [2]:
username = "rbsdq_odpp@ROSBANK.RUS.SOCGEN"

p = subprocess.Popen(f"/usr/bin/id {username} -u",
                     shell=True,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)

id_str1 = p.stdout.readline().decode("utf-8").replace("\n", "")
p = subprocess.Popen(f"id  -u",
                     shell=True,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)
id_str2 = p.stdout.readline().decode("utf-8").replace("\n", "")

os.environ ["JAVA_TOOL_OPTIONS"] = "-Djava.security.krb5.conf=/etc/krb5.conf.d/krb5.conf"
os.environ ["KRB5_CONFIG"] = "/etc/krb5.conf.d/krb5.conf"
os.environ ["KRB5CCNAME"] = f"/tmp/krb5cc_{id_str1}_{id_str2}"

spark_home = "/usr/hdp/current/spark2-client/"
os.environ ['SPARK_HOME'] = spark_home

sys.path.insert(0, os.path.join(spark_home, 'python'))
os.environ['PYSPARK_PYTHON'] = './environment/bin/python3.7'
os.environ ['HADOOP_CONF_DIR'] = '/usr/hdp/current/hadoop-client/conf/'
os.environ ['PYTHONPATH'] = '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip:/usr/hdp/current/spark2-client/python/'
os.environ ['PYTHONSTARTUP'] = '/usr/hdp/current/spark2-client/python/pyspark/shell.py'
os.environ['PYSPARK_SUBMIT_ARGS'] = '''--master yarn --deploy-mode client 
--num-executors 32
--executor-cores 1
--driver-memory 12g  
--conf spark.driver.maxResultSize=12g 
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--conf spark.sql.execution.arrow.pyspark.enabled=true
--conf spark.sql.crossJoin.enabled=true
--executor-memory 8g
--conf spark.driver.cores=1
--conf spark.executor.memoryOverhead=2048
--conf spark.executor.extraJavaOptions=-XX:+UseG1GC
--conf spark.yarn.dist.archives=hdfs:///datalab/python/python-3.7-dev/python3-venv.tar.gz#environment 
pyspark-shell'''

exec(open(os.path.join(spark_home,'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2.3.1.4.0-315
      /_/

Using Python version 3.7.4 (default, Jul 21 2020 23:48:23)
SparkSession available as 'spark'.


In [5]:
class DataFrameMemorySize:
    """Класс для расчета размера датафрейма в памяти"""
    
    TABLE_MEMORY_LIMIT_BYTES = 52488000 # 50mb максимально допустимый порог веса одной таблицы
    COLUMN_MEMORY_LIMIT_BYTES = 524880 # 0.5mbмаксимально допустимый порог веса одной колонки таблицы
    ROWS_LIMIT = 5000 # размер выборки
    
    
    def __init__(self, schema_name: str, table_name: str) -> None:
        self.source_name = f'{schema_name}_{table_name}'
        self.source_table = spark.table(f'{schema_name}.{table_name}')
        self.cached_limit_df = self.cacheLimitTable()
        
       
    @staticmethod
    def createCachedNamedDF(df: pyspark.sql.dataframe.DataFrame, name: str) -> None:
        """Функция для создания кэширования датафрейма"""
        
        df.createOrReplaceTempView(name)
        spark.catalog.cacheTable(name)
        df.count()
    
    
    def sourceColumn(self, colum_name: str) -> str:
        """Функция для получения столбца целевой таблицы"""
        
        return f'{self.source_name}_{colum_name}_column'
        
        
    def cacheLimitTable(self) -> None:
         """Функция для кэширования таблицы Hive"""
        
        cached_table = (self.source_table
            .limit(self.ROWS_LIMIT))
        self.createCachedNamedDF(cached_table, self.source_name)
        return cached_table
        
        
    def cacheLimitColumn(self, column_name: str) -> None:
        """Функция для создания кэширования столбца таблицы Hive"""
        
        cached_column = (self.cached_limit_df
            .select(column_name))
        source_column = self.sourceColumn(column_name)
        self.createCachedNamedDF(cached_column, source_column)

        
    def avgColumnSize(self, table_memory_size: int) -> float:\
        """Функция для расчета среднего размера столбца в памяти"""
        
        return table_memory_size / len(self.source_table.columns)
    
    
    def getColumnsToCheck(self) -> List[str]:
        """Функция для получения столбцов для проверки"""
        
        data_types_to_check   = ['string', 'varchar', 'char', 'binary']
        data_types_df = pd.DataFrame(self.cached_limit_df.dtypes, columns=['column_name', 'type'])
        columns_to_check = (data_types_df
            .query('type in @data_types_to_check')
            .column_name
            .tolist())
        return columns_to_check
        
    
    @staticmethod
    def getCachedDataFrameFromStorageByName(df_name: str) -> py4j.java_gateway.JavaObject:
        """Функция для получения объекта кэшированного датафрейма из хранилища Spark"""
        
        spark_memory_storage = sc._jsc.sc().getRDDStorageInfo() 
        for cached_df in spark_memory_storage:
            cached_df_name = cached_df.name().replace('In-memory table ', '')
            if cached_df_name == df_name:
                return cached_df


    @staticmethod
    def caclulateDataFrameMemorySize(cached_df: py4j.java_gateway.JavaObject) -> int:
        """Функция для расчета размера в памяти датафрейма"""
        
        memory_size = cached_df.memSize() 
        disk_size = cached_df.diskSize()
        if disk_size > 0:
            raise DiskMemoryUsageError
        return memory_size
    
    
    @classmethod
    def getDataFrameSizeInBytes(self, df_name: str) -> int:
        """Функция для получения размера датафрейма в байтах"""
        
        cached_df = self.getCachedDataFrameFromStorageByName(df_name)
        df_memory_size = self.caclulateDataFrameMemorySize(cached_df)
        return df_memory_size
    
    
    def isAnomalColumnMemorySize(self, column: str) -> bool:
        """Функция для проверки на превышение допустимого размера в памяти кэшированного столбца"""
        
        self.cacheLimitColumn(column)
        source_column = self.sourceColumn(column)
        column_memory_size = self.getDataFrameSizeInBytes(source_column) 
        is_anomal_column = column_memory_size > self.COLUMN_MEMORY_LIMIT_BYTES
        return is_anomal_column
    
    
    def getAnomalColumnsByMemorySize(self) -> List[str]:
        """Функция для столцев датафрейма, превышающих допустимый размер"""
        
        columns_to_check = self.getColumnsToCheck()
        anomal_columns = [column for column in columns_to_check if self.isAnomalColumnMemorySize(column)]
        return anomal_columns
    
    
    def isAnomalTableByMemorySize(self) -> bool:  
        """Функция для проверки на превышение допустимого размера в памяти кэшированной таблицы"""
        
        table_memory_size = self.getDataFrameSizeInBytes(self.source_name)
        avg_column_size = self.avgColumnSize(table_memory_size)
        is_anomal_table_size = table_memory_size > self.TABLE_MEMORY_LIMIT_BYTES 
        is_anomal_avg_column_size = avg_column_size > self.COLUMN_MEMORY_LIMIT_BYTES 
        is_anomal_table = is_anomal_table_size or is_anomal_avg_column_size  
        
        return is_anomal_table
    
        
    def checkMemorySize(self) -> List[str]:
        """Основная функция для проверки датафрейма"""
        
        try:
            if self.isAnomalTableByMemorySize():           
                except_columns = self.getAnomalColumnsByMemorySize()
            else:
                except_columns = []      
        except DiskMemoryUsageError:
            except_columns = self.getBigColumnsByMemorySize()          
        
        self.cached_limit_df.unpersist()
        return except_columns   
    

class DetectingAnomalColumns:
    """Класс для поиска аномальных столбцов в таблице по размеру в памяти"""
    
    
    @staticmethod
    def getSourceTables() -> pd.DataFrame:
        """Функция для получения таблиц, которые необходимо проверить"""
        
        query = """
        SELECT i.schema_name, i.table_name
        FROM dq_sbx.info i
        WHERE i.is_in_archive = 'false'
        GROUP BY i.schema_name, i.table_name
        """
        source_tables_df = get_df_from_dq_profiling(query)
        return source_tables_df
    
    
    @staticmethod
    def toSQL(schema: str, table: str, columns: List[str]) -> str:
        """Функция для загрузки в БД"""
        
        insert_query = 'INSERT INTO dq_sbx.blacklist_columns_by_memory_size VALUES ' 
        sampling_date = datetime.now().strftime('%Y-%m-%d')
        values = [str(tuple([schema, table, column, sampling_date])).lower() for column in columns]
        insert_query = insert_query + ', '.join(values)
        return insert_query 
    
    
    @classmethod
    def loadAnomalColumns(cls, source) -> None:
        """Функция для поиска и загрузки аномальных колонок"""
        
        try:
            anomal_columns = DataFrameMemorySize(source.schema_name, source.table_name).checkMemorySize()
            if anomal_columns:
                query = cls.toSQL(source.schema_name, source.table_name, anomal_columns)
                execute_postgres_dq_profiling(query)
            else:
                pass
        except Exception:
            pass


    @classmethod
    def detect(cls) -> None:
        """Основная функция для поиска колонок с большим весом в памяти"""
        try:
            truncate_table = 'TRUNCATE dq_sbx.blacklist_columns_by_memory_size'
            execute_postgres_dq_profiling(truncate_table)
        except Exception:
            pass
        source_tables_df = cls.getSourceTables()
        with ThreadPoolExecutor(16) as pool:
            pool.map(cls.loadAnomalColumns, source_tables_df.itertuples(index=False))

In [6]:
%%time
DetectingAnomalColumns.detect()

CPU times: user 30 s, sys: 7.8 s, total: 37.8 s
Wall time: 32min 18s
